In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

import os, sys, pickle, copy, pygmt, operator

from pathlib import Path
from time import sleep
from tqdm import tqdm, tqdm_notebook

import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.gridspec as gridspec
from matplotlib import pyplot as plt

import seaborn as sns; sns.set()
sns.set_theme(style="white")

#from joblib import Parallel, delayed
#import multiprocessing
#import numba as nb
#from numba import jit

from sklearn.feature_selection import RFECV,RFE
from sklearn.model_selection import train_test_split, KFold, cross_validate, cross_val_score

from sklearn.metrics import make_scorer, explained_variance_score, mean_absolute_error, mean_squared_error, \
r2_score, max_error, median_absolute_error, mean_absolute_percentage_error

from sklearn.preprocessing import PowerTransformer, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.compose import make_column_selector as selector
from sklearn.inspection import permutation_importance


In [ ]:
# constanst
crs_to='epsg:4326'
crs_from='epsg:4326'
projection = 'M5.4i'
#parent directory

DIR = Path().resolve() 

plt_params = {
    'figure.titlesize' : 28,
    "axes.titlesize" : 25, # main
    "axes.labelsize" : 25,  # labels
    "axes.edgecolor" : "black", 
    "axes.linewidth" : 1, 
    'xtick.labelsize': 20, # ticks
    'ytick.labelsize': 20,
    'legend.title_fontsize':17,
    'legend.fontsize': 17,
    'font.size': 20,
}

# We can exclude Arctic ocean and Antarctica, as there are no HF measurements to use
world_lon_min, world_lon_max, world_lat_min, world_lat_max  = -180, 180, -60, 80

# map extents of Africa and Australia
afr_lon_min, afr_lon_max, afr_lat_min, afr_lat_max =  -20, 52, -37 , 38  




In [ ]:

class SeabornFig2Grid():

    def __init__(self, seaborngrid, fig,  subplot_spec):
        self.fig = fig
        self.sg = seaborngrid
        self.subplot = subplot_spec
        if isinstance(self.sg, sns.axisgrid.FacetGrid) or \
            isinstance(self.sg, sns.axisgrid.PairGrid):
            self._movegrid()
        elif isinstance(self.sg, sns.axisgrid.JointGrid):
            self._movejointgrid()
        self._finalize()

    def _movegrid(self):
        """ Move PairGrid or Facetgrid """
        self._resize()
        n = self.sg.axes.shape[0]
        m = self.sg.axes.shape[1]
        self.subgrid = gridspec.GridSpecFromSubplotSpec(n,m, subplot_spec=self.subplot)
        for i in range(n):
            for j in range(m):
                self._moveaxes(self.sg.axes[i,j], self.subgrid[i,j])

    def _movejointgrid(self):
        """ Move Jointgrid """
        h= self.sg.ax_joint.get_position().height
        h2= self.sg.ax_marg_x.get_position().height
        r = int(np.round(h/h2))
        self._resize()
        self.subgrid = gridspec.GridSpecFromSubplotSpec(r+1,r+1, subplot_spec=self.subplot)

        self._moveaxes(self.sg.ax_joint, self.subgrid[1:, :-1])
        self._moveaxes(self.sg.ax_marg_x, self.subgrid[0, :-1])
        self._moveaxes(self.sg.ax_marg_y, self.subgrid[1:, -1])

    def _moveaxes(self, ax, gs):
        #https://stackoverflow.com/a/46906599/4124317
        ax.remove()
        ax.figure=self.fig
        self.fig.axes.append(ax)
        self.fig.add_axes(ax)
        ax._subplotspec = gs
        ax.set_position(gs.get_position(self.fig))
        ax.set_subplotspec(gs)

    def _finalize(self):
        plt.close(self.sg.fig)
        self.fig.canvas.mpl_connect("resize_event", self._resize)
        self.fig.canvas.draw()

    def _resize(self, evt=None):
        self.sg.fig.set_size_inches(self.fig.get_size_inches())

In [ ]:
obs = pd.DataFrame()


obs["OBS_REF"] = ["CTD" ,  "SI","LAB", "MOHO",
            "SV","PV", 
            "GEOID","FA","DEM","BG", "EMAG2_CLASS",
                   "RHO_L", "RHO_C", 
                  "VOLC_DIST_W", "REG", "GLIM"]

obs["OBS_AFR"] = ["CTD" ,  "SI","LAB", "MOHO",
            "SV_Velocity","PV_Velocity", 
            "GEOID","FA","DEM","BG", "EMAG2",
                   "RHO_L", "RHO_C", 
                  "VOLC_DIST", "REG", "GLIM"]
  
     
# Labels for plots etc


# Labels for plots etc
obs["LABELS_gmt"] = ["CTD",  "Shape index", "LAB", "Moho", 
                "S@_v@ 150km", "P@_v@ 150km", 
                "Geoid", "Free air", "DEM", "Bouguer", "Mag.", 
                "Lith. rho", "Crust rho",  
                 "Volcano", "REG", "GliM", ]  


obs["LABELS"] = ["CTD",  "Shape index", "LAB", "Moho", 
                "$S_v$ @150km", "$P_v$ @150km", 
                "Geoid", "Free air", "DEM", "Bouguer", "Mag.", 
                "Lith. ρ", "Crust ρ",  
                 "Volcano", "REG", "GliM", ]
    
# "vp/vs"
# Units to display in plots etc
obs["UNITS"] = ["km",  "si", "km", "km",
             "$\delta$$S_v$ %","$\delta$$P_v$ %", 
             "m", "mGal", "m", "mGal",  "f(nT)", 
                 "kg/m$^3$", "kg/m$^3$",
                "km",  "class", "class"]



obs["UNITS_gmt"] = ["km",  "si", "km", "km",
             "km/s","km/s", 
             "m", "mGal", "m", "mGal",  "f(nT)", 
                 "kg/m@+3@+", "kg/m@+3@+",
                "km",  "class", "class"]
        
# Range of colormap for plots. Similar data are placed in same ranges for consistancy
obs["V_RANGE"] = [(0,50), (-1,1),(0,300),(15,60),
              (-0.075,0.075), (-0.02,0.02), 
              (-45,45), (-100,100) , (-2200, 2200),(-100,100),  (-0.4, 0.4), 
                   (3260, 3360), (2650, 2950),
                  (0,1), (1,6),(1,16),]


    
obs["V_RANGE_AFR"] = [(0,50), (-1,1),(50,250),(20,50),
          (-0.075,0.075), (-0.02,0.02), 
          (-45,45), (-100,100) , (-2200, 2200),(-100,100),  (-200, 200), 
               (3260, 3360), (2650, 2950),
              (0,100), (1,6),(1,15),]


obs["CMAPS"] = ["batlow",  "broc", "bamako", "batlow", 
             "roma","roma", 
             "bamako", "broc", "bukavu", "broc", "batlow",            
                "batlow", "batlow",
               "bamako",  "batlowS","categorical", ]
'''
obs["CMAPS"] = ["SCM/bamako",  "SCM/broc", "SCM/bamako", "SCM/bamako", 
             "SCM/roma","SCM/roma", 
             "SCM/bamako", "SCM/broc", "SCM/oleron", "SCM/broc", "SCM/bilbao",            
                "SCM/batlow", "SCM/batlow",
               "SCM/broc",  "gmt/categorical","gmt/categorical", ]'''


new_index = [0,1,2,3,4,5,6,8,7,9,10,11,12,13,14,15]

#new_index = [4,3,15,6,7,0, 14, 10,16, 8, 9,2, 13, 12, 8, 11, ]

obs = obs.reindex(new_index)

#obs.index = np.arange(0,len(obs))

pd.options.display.width = 370
pd.options.display.max_colwidth = 16




obs_dict = obs.to_dict(orient='records')

obs.set_index(['OBS_REF'], inplace=True)



obs

In [ ]:
ds_afr = xr.load_dataset(DIR/'Grids'/'Inputs'/"ds_afr.nc")
#ds_world = xr.load_dataset(dir_p/'Grids'/'inputs'/"ds_world.nc")



In [ ]:

target = 'GHF'
coord = ['lon', 'lat']
grid_index_world = 'grid_index_world'
grid_index_afr ='grid_index_afr'

hq_lower_bound = 10
hq_upper_bound = 200


#######

features_ex = []
features_ghf = []



features = obs.index.to_list()



in_features = set(features)

features_ex = copy.deepcopy(features)
features_ex.extend(coord)
features_ex.append(grid_index_world)
features_ex.append(grid_index_afr)

features_ex.append(target)

features_ghf = copy.deepcopy(features)
features_ghf.append(target)


features_ex

In [ ]:


rating = 'rab'
outlier = 'OD'


file_label = f'{outlier}_{rating}'

w_OD_rab_f =  DIR/'Dataset'/'Preprocessed'/f'W_OD_rab.csv'
W_OD_rab = pd.read_csv(w_OD_rab_f, sep='\t')

#mask_afr = W_OD_rab['lon'].between(afr_lon_min, afr_lon_max)& W_OD_rab['lat'].between(afr_lat_min, afr_lat_max)
#Afr_OD_rab = W_OD_rab[~mask_afr]

Afr_OD_rab_f =  DIR/'Dataset'/'Preprocessed'/f'Afr_OD_rab.csv'
Afr_OD_rab = pd.read_csv(Afr_OD_rab_f,  sep='\t')


Afr_NOD_rab_f =  DIR/'Dataset'/'Preprocessed'/f'Afr_NOD_rab.csv'
Afr_NOD_rab = pd.read_csv(Afr_NOD_rab_f,sep='\t')


#######
W_OD_rab['GLIM']  = W_OD_rab['GLIM'].astype('category')
W_OD_rab['REG']  = W_OD_rab['REG'].astype('category')

Afr_OD_rab['GLIM']  = Afr_OD_rab['GLIM'].astype('int').astype('category')
Afr_OD_rab['REG']   = Afr_OD_rab['REG'].astype('int').astype('category')

Afr_NOD_rab['GLIM']  = Afr_NOD_rab['GLIM'].astype('int').astype('category')
Afr_NOD_rab['REG']   = Afr_NOD_rab['REG'].astype('int').astype('category')


#####

X_w = W_OD_rab[features]
y_w = W_OD_rab[target].values.reshape(-1,1) 


X_afr = Afr_OD_rab[features]
y_afr = Afr_OD_rab[target].values.reshape(-1,1) 




bs_rfr_hyp =  DIR/'Hyperparameters'/f'BS_hyperparameter.csv'



bs_rfr_hyp_df = pd.read_csv(bs_rfr_hyp, sep='\t')


best_params = bs_rfr_hyp_df.to_dict('r')[0]

print(file_label)


print(len(Afr_OD_rab))
print(len(W_OD_rab))


X_afr.describe(include='all')

In [ ]:
def concordance_correlation_coefficient(y_true, y_pred):
    """Concordance correlation coefficient."""
    
    y_true = y_true.ravel().reshape(-1,)
    y_pred = y_pred.ravel().reshape(-1,)
    # Remove NaNs
    df = pd.DataFrame({
        'y_true': y_true,
        'y_pred': y_pred
    })
    df = df.dropna()
    y_true = df['y_true']
    y_pred = df['y_pred']
    # Pearson product-moment correlation coefficients
    cor = np.corrcoef(y_true, y_pred)[0][1]
    # Mean
    mean_true = np.mean(y_true)
    mean_pred = np.mean(y_pred)
    # Variance
    var_true = np.var(y_true)
    var_pred = np.var(y_pred)
    # Standard deviation
    sd_true = np.std(y_true)
    sd_pred = np.std(y_pred)
    # Calculate CCC
    numerator = 2 * cor * sd_true * sd_pred
    denominator = var_true + var_pred + (mean_true - mean_pred)**2
    return numerator / denominator

def nrmse(y_true, y_pred):
    cost  = mean_squared_error(y_true, y_pred, squared=False)
    return cost/(y_true.mean()) 

def accuracy(y_true, y_pred):
    cost  = mean_absolute_percentage_error(y_true, y_pred)
    return (100 - cost) * 100

def min_e(y_true, y_pred):
    cost  = abs(y_test - y_predict)
    return cost.min()

def mpe(y_true, y_pred):
    return (np.mean((y_test -y_predict)/y_test) , 2)


rmse_score = make_scorer(mean_squared_error , squared=False, greater_is_better=False)

scores_cv = {

'RMSE'     :  rmse_score,
#'NRMSE'    :  make_scorer(nrmse),
'MAE'      :  make_scorer(mean_absolute_error),    
#'MAPE'     :  make_scorer(mean_absolute_percentage_error ),
#'ACC'      :  make_scorer(accuracy) ,
#'MPE'      :  make_scorer(mpe),
'CD'       :  make_scorer(r2_score),
#'EV'       :  make_scorer(explained_variance_score),
#'MAX_E'    :  make_scorer( max_error),
#'MIN_E'    :  make_scorer(min_e),
#'MedAE'    :  make_scorer(median_absolute_error),
#'CCC'      :  make_scorer(concordance_correlation_coefficient),
    
}


In [ ]:
#rfecv this process is more stable
 
KFOLD=3
N_ITER=30
RANDOM_STATE=42
N_JOBS=1
VERBOSE=2
results_cv ={}


CV = KFold(n_splits=KFOLD, random_state=RANDOM_STATE, shuffle=True)


#scoring = make_scorer(nrmse , greater_is_better=False )
nrmse_score = make_scorer(nrmse , greater_is_better=False)
#r2 = make_scorer(r2_score)
#scoring = make_scorer(nrmse , greater_is_better=False )
#ccc_score = make_scorer(concordance_correlation_coefficient , greater_is_better=True)

scorings = {
    'NRMSe':nrmse_score,
    #'RMSE':'neg_root_mean_squared_error', 
    #'MAE':'neg_mean_absolute_error', 
    #'MAPE':'neg_mean_absolute_percentage_error', 
    'R2':'r2',
    #'EV':'explained_variance'   
    #'$p_c$': ccc_score,    
}




tuned_params = {item.replace('ttr__regressor__', 'rfecv__estimator__'): best_params[item] for item in best_params}




for key, score in scorings.items():
    
    regressor = RandomForestRegressor(random_state=RANDOM_STATE)


    numeric_transformer = PowerTransformer(method='yeo-johnson',standardize=True)

    categorical_transformer = OrdinalEncoder(handle_unknown='use_encoded_value' , unknown_value =-1)


    preprocessor = ColumnTransformer(
        transformers=[
       ("num", numeric_transformer, selector(dtype_exclude="category")),
            ("cat", categorical_transformer, selector(dtype_include="category")),
        ]
    )
    
    rfecv = RFECV(estimator=regressor, cv=CV, step=1, n_jobs=N_JOBS, 
                             scoring=score, verbose=VERBOSE)

    # Append classifier to preprocessing pipeline.
    # Now we have a full prediction pipeline.
    steps=[("preprocessor", preprocessor), 
           ("rfecv", rfecv)]


    # Initialize Pipeline object
    pipeline= Pipeline(steps=steps)
    pipeline.set_params(**tuned_params)



    # Initialize RFECV object
    #feature_selector = RFECV(pipeline['regressor'], cv = cv, step = 1, #n_jobs=-1, 
    #                         scoring = score, verbose = 1)
    

    # Fit RFECV
    pipeline.fit(X_afr, y_afr)
    #feature_selector .fit(X_w, np.ravel(y_w))
    results_cv[key] = pipeline

    # Get selected features
    #feature_names = X_train_corr.columns
    #selected_features = feature_names[feature_selector.support_].tolist()

    print(f'terminated {key} {score}\n')
    print('terminated')



In [ ]:
# get RFECV progression score
score_progression = pd.DataFrame()
score_progression["NRMSe_RFECV"] = results_cv['NRMSe']['rfecv'].cv_results_['mean_test_score'].round(3) * -1
score_progression["R2_RFECV"] = results_cv['R2']['rfecv'].cv_results_['mean_test_score'].round(3)
score_progression["n_features"]= list(range(1,17))
score_progression = score_progression.set_index("n_features")
score_progression

In [ ]:
#feature importance 
# Get selected features data set
# should be scaled

sub_figs = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')

# Set figure size and create barplot
#sns.set(style="whitegrid", color_codes=True, font_scale = 1.8)
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(27, 6))

plt.rcParams.update(plt_params)



feature_importance = pd.DataFrame()
 

# Load hyper parameter 

numeric_transformer = PowerTransformer(method='yeo-johnson', standardize=True)
categorical_transformer = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

preprocessor = ColumnTransformer(
    transformers=[
           ("num", numeric_transformer, selector(dtype_exclude="category")),
           ("cat", categorical_transformer, selector(dtype_include="category")),
    ], 
    #remainder='passthrough', verbose_feature_names_out=False
)


regressor = RandomForestRegressor(random_state=RANDOM_STATE)


steps=[("preprocessor", preprocessor), 
       #("regressor", regressor), 
       ('ttr', TransformedTargetRegressor(regressor=regressor, transformer=numeric_transformer))
      ]

# Initialize Pipeline object
pipeline = Pipeline(steps=steps)

pipeline.set_params(**best_params)


pipeline.fit(X_afr,y_afr)


#####

feature_importance["RI"] = pipeline['ttr'].regressor_.feature_importances_.round(2)
feature_importance["LABELS"]= obs.loc[features , 'LABELS'].values
feature_importance["OBS_REF"] = obs.index
feature_importance.set_index('LABELS', inplace=True)


permutaion_ranking = permutation_importance(pipeline, X_afr, y_afr, n_repeats=10,
                                    scoring=rmse_score, random_state=RANDOM_STATE)
perm_sorted_idx = permutaion_ranking.importances_mean.argsort()


obs.reset_index(drop=False).set_index('OBS_REF', inplace=True)

# Get feature importance


# Get feature importance
feature_importance["PFI"] = permutaion_ranking.importances_mean


feature_importance = feature_importance.sort_values(by="RI", ascending=True)
ax1.barh(feature_importance.index, feature_importance['RI'], height=0.7)


feature_importance = feature_importance.sort_values(by="PFI", ascending=True)
ax2.barh(feature_importance.index, feature_importance['PFI'], height=0.7)

ax3.boxplot(
    permutaion_ranking.importances[perm_sorted_idx].T,
    vert=False,
    labels=obs.loc[X_afr.iloc[:,perm_sorted_idx].columns , 'LABELS'].values,
)



# Tight layout
plt.tight_layout()

plt.show()

#fig.savefig(DIR/'fig'/f"fig_8_PFI_rab.jpg", bbox_inches='tight', dpi=300 )


# Save Figure
#fig.savefig(dir_p/ 'fig'/"fig_5.jpeg", bbox_inches='tight', dpi=300 )

In [ ]:
# individual RFECV to calculate conribution of each feature to R2 and NRMSe

for feature in obs.index:

    print(feature)


    X_train_afr_feature = X_afr[feature].to_frame()

    numeric_transformer = PowerTransformer(method='yeo-johnson', standardize=True)
    categorical_transformer = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

    preprocessor = ColumnTransformer(
        transformers=[
               ("num", numeric_transformer, selector(dtype_exclude="category")),
               ("cat", categorical_transformer, selector(dtype_include="category")),
        ], 
        #remainder='passthrough', verbose_feature_names_out=False
    )


    regressor = RandomForestRegressor(random_state=RANDOM_STATE)

    # Append classifier to preprocessing pipeline.
    # Now we have a full prediction pipeline.

    steps=[("preprocessor", preprocessor), 
           #("regressor", regressor), 
           ('ttr', TransformedTargetRegressor(regressor=regressor, transformer=numeric_transformer))
          ]

    # Initialize Pipeline object
    pipeline = Pipeline(steps=steps)

    pipeline.set_params(**best_params)



    for key, score in scorings.items():
        feature_importance.loc[obs.loc[feature, 'LABELS'], key] = np.mean(cross_validate(
            pipeline, 
            X_train_afr_feature, 
            y_afr,
            scoring=score,
              cv=CV)['test_score'])


feature_importance[['RI', 'PFI', 'NRMSe', 'R2']] = feature_importance[['RI', 'PFI', 'NRMSe', 'R2']].abs() 

print('terminated')

In [ ]:
feature_importance = feature_importance.sort_values(by="RI", ascending=False)

feature_importance["CUMSUM"] = feature_importance["RI"].cumsum()
score_progression["CUMSUM"] = feature_importance["CUMSUM"].values

mask_RI_NRMSE = (feature_importance['RI'] < feature_importance['NRMSe']) & (feature_importance['NRMSe'] < feature_importance['R2'])

mask_NRMSE_RI = (feature_importance['NRMSe'] < feature_importance['RI']) & (feature_importance['RI'] < feature_importance['R2']) 
    
mask_R2_RI = (feature_importance['R2'] < feature_importance['RI']) & (feature_importance['RI'] < feature_importance['NRMSe'])


mask_RI_R2 = (feature_importance['RI'] < feature_importance['R2']) & (feature_importance['R2'] < feature_importance['NRMSe'])

mask_NRMSE_R2 = (feature_importance['NRMSe'] < feature_importance['R2']) & (feature_importance['R2'] < feature_importance['RI']) 


mask_R2_NRMSE = (feature_importance['R2'] < feature_importance['NRMSe']) & (feature_importance['NRMSe'] < feature_importance['RI'])



#feature importance 
# Get selected features data set
# should be scaled
fig, axes = plt.subplots(1,2, figsize=(19,8),gridspec_kw={'width_ratios': [3, 2.5]})
sns.set(style="whitegrid")


plt.rcParams.update(plt_params)


axes.ravel()
#sns.set(style="whitegrid", color_codes=True, font_scale = 3)



###1
if mask_RI_NRMSE.any():
    sns.barplot(x = feature_importance['R2'].values * mask_RI_NRMSE.astype(int).values, 
                y = feature_importance.index, 
                ax=axes[0], color='lightblue',
                )


    sns.barplot(x = feature_importance['NRMSe'].values * mask_RI_NRMSE.astype(int).values, 
                y = feature_importance.index, 
                ax=axes[0], color='orange', label='NRMSe',
                )
    sns.barplot(x = feature_importance["RI"].values * mask_RI_NRMSE.astype(int).values, 
                y = feature_importance.index,  
                ax=axes[0],  color='lightgreen',label=f"""Relative
    importance"""
                        )

###2
if mask_NRMSE_RI.any():
    sns.barplot(x = feature_importance['R2'].values * mask_NRMSE_RI.astype(int).values, 
                y = feature_importance.index, 
                ax=axes[0], color='lightblue',
                )


    sns.barplot(x = feature_importance["RI"].values * mask_NRMSE_RI.astype(int).values, 
                y = feature_importance.index,  
                ax=axes[0],  color='lightgreen',label=f"""Relative
    importance"""
                        )

    sns.barplot(x = feature_importance['NRMSe'].values * mask_NRMSE_RI.astype(int).values, 
                y = feature_importance.index, 
                ax=axes[0], color='orange', label='NRMSe',
                )
    

###3
if mask_R2_RI.any():
    sns.barplot(x = feature_importance['NRMSe'].values * mask_R2_RI.astype(int).values, 
                y = feature_importance.index, 
                ax=axes[0], color='orange', label='NRMSe',
                )


    sns.barplot(x = feature_importance["RI"].values * mask_R2_RI.astype(int).values, 
                y = feature_importance.index,  
                ax=axes[0],  color='lightgreen',label=f"""Relative
    importance"""
                        )


    
    sns.barplot(x = feature_importance['R2'].values * mask_R2_RI.astype(int).values, 
            y = feature_importance.index, 
            ax=axes[0], color='lightblue',
            )
    

###4
if mask_RI_R2.any():
    
    sns.barplot(x = feature_importance['NRMSe'].values * mask_RI_R2.astype(int).values, 
                y = feature_importance.index, 
                ax=axes[0], color='orange', label='NRMSe',
                )
    
    sns.barplot(x = feature_importance['R2'].values * mask_RI_R2.astype(int).values, 
                y = feature_importance.index, 
                ax=axes[0], color='lightblue',
                )

    sns.barplot(x = feature_importance["RI"].values * mask_RI_R2.astype(int).values, 
                y = feature_importance.index,  
                ax=axes[0],  color='lightgreen',label=f"""Relative
    importance"""
                        )

###5
    
if mask_NRMSE_RI.any():
    sns.barplot(x = feature_importance['R2'].values * mask_NRMSE_RI.astype(int).values, 
                y = feature_importance.index, 
                ax=axes[0], color='lightblue',
                )


    sns.barplot(x = feature_importance["RI"].values * mask_NRMSE_RI.astype(int).values, 
                y = feature_importance.index,  
                ax=axes[0],  color='lightgreen',label=f"""Relative
    importance"""
                        )

    sns.barplot(x = feature_importance['NRMSe'].values * mask_NRMSE_RI.astype(int).values, 
                y = feature_importance.index, 
                ax=axes[0], color='orange', label='NRMSe',
                )
    
####6
if mask_R2_RI.any():
    
   
    sns.barplot(x = feature_importance['NRMSe'].values * mask_R2_RI.astype(int).values, 
                y = feature_importance.index, 
                ax=axes[0], color='orange', label='NRMSe',
                )
    sns.barplot(x = feature_importance["RI"].values * mask_R2_RI.astype(int).values, 
                y = feature_importance.index,  
                ax=axes[0],  color='lightgreen',label=f"""Relative
    importance"""
                        )
    
    sns.barplot(x = feature_importance['R2'].values * mask_R2_RI.astype(int).values, 
            y = feature_importance.index, 
            ax=axes[0], color='lightblue',
            )
    
    
### optimal

'''
if mask_RI_NRMSE.any():
    sns.barplot(x = feature_importance.loc[mask_RI_NRMSE[0:11], 'R2'].values, 
                y = feature_importance.loc[mask_RI_NRMSE[0:11], 'R2'].index, 
                ax=axes[0], color='lightblue',
                )


    sns.barplot(x = feature_importance.loc[mask_RI_NRMSE[0:11],'NRMSe'].values, 
                y = feature_importance.loc[mask_RI_NRMSE[0:11],'NRMSe'].index, 
                ax=axes[0], color='orange', label='NRMSe',
                )
    sns.barplot(x = feature_importance.loc[mask_RI_NRMSE[0:11],"RI"].values, 
                y = feature_importance.loc[mask_RI_NRMSE[0:11],"RI"].index,  
                ax=axes[0],  color='lightgreen',label=f"""Relative
    importance"""
                        )
    
''' 


axes[0].set_ylabel('')
axes[0].grid(False)



#axes[0].patch.set_edgecolor('black')  
#axes[0].patch.set_linewidth('3') 
# Hide the right and top spines
axes[0].spines.right.set_visible(False)
axes[0].spines.top.set_visible(False)

# Only show ticks on the left and bottom spines
axes[0].yaxis.set_ticks_position('left')
axes[0].xaxis.set_ticks_position('bottom')





axes[0].locator_params(axis='x', nbins=7)
#axes[0].legend(loc=[.55, .6], framealpha=0.5)

axes[0].set_title(f'{sub_figs[0]})   Ranking of Observables', loc ='left', pad=10, size=25, x=-0.05, y=1.05)
axes[0].set_xlabel("Normalized Score")

markersize = 12
linewidth = 3
for key, score in scorings.items():
    #selected_features = feature_names[results[key].support_].tolist()
    # Get Performance Data
    #print(f"Optimal number of features for {key} : {results[key].n_features_}")
    if key == 'NRMSe':
        #results[key].support_rfecv_df = pd.DataFrame(results[key].ranking_,index=X_afr_lr.columns,columns=['Rank']).sort_values(by='Rank',ascending=True)

        axes[1].plot(score_progression.index, 
                 score_progression['NRMSe_RFECV'], color='orange', linewidth=linewidth,
                   marker="^", label='$NRMSe$', markersize=markersize,)


    else:
        axes[1].plot(score_progression.index,
                 score_progression['R2_RFECV'], color='lightblue', 
                     linewidth=linewidth, marker="d",  label='$R^2$', markersize=markersize,
                       )


axes[1].plot(score_progression.index, 
         score_progression['CUMSUM'], linewidth=linewidth, 
            color='lightgreen',  marker="o", 
             label=f'''Importance''', markersize=markersize,)


axes[1].set_xlabel("Number of observables")

axes[1].set_ylabel("Normalized Score")
axes[1].legend(loc=[.45, .23], frameon=False, fontsize=22, )
#axes[1].legend(loc=[.01, .7])
#plt.axesvline(results[key].n_features_ ,color='r')
axes[1].set_title(f'{sub_figs[1]})   Progression of Scores and Misfit', loc ='left', pad=10, size=25,  x=-0.05, y=1.05)
axes[1].locator_params(axis='y', nbins=10)
axes[1].locator_params(axis='x', nbins=9)
axes[1].grid(False)

axes[1].spines.right.set_visible(False)
axes[1].spines.top.set_visible(False)

# Only show ticks on the left and bottom spines
axes[1].yaxis.set_ticks_position('left')
axes[1].xaxis.set_ticks_position('bottom')


plt_params = {
    #'figure.titlesize' : 28,
    "axes.titlesize" : 22, # main
    "axes.labelsize" : 22,  # labels
    "axes.edgecolor" : "black", 
    "axes.linewidth" : 1.2, 
    'xtick.labelsize': 22, # ticks
    'ytick.labelsize': 22,
    #'legend.fontsize': 20,
     'legend.handlelength': 5,
    #'legend.title_fontsize':5,
    #'legend.fontsize': 5,
    #'font.size': 18,
}

plt.rcParams.update(plt_params)

# Tight layout
plt.tight_layout()


# Save Figure
#fig.savefig(dir_p/'fig'/'presentation'/"fig_p6d.jpeg", bbox_inches='tight', dpi=300 )
fig.savefig(DIR/'fig'/"fig_5.pdf", bbox_inches='tight', dpi=300 )
fig.savefig(DIR/'fig'/"fig_5.jpg", bbox_inches='tight', dpi=300 )

####




# Modeling

In [ ]:
#target values
AFR_grid = pd.DataFrame({'X': ds_afr.XV.values.ravel(), 'Y': ds_afr.YV.values.ravel()})
for feature in tqdm_notebook(features ,  desc=f'Processing: ', leave=False ):
    sleep(0.01)
    AFR_grid[feature] = ds_afr[feature].values.ravel()

AFR_grid['GLIM']  = AFR_grid['GLIM'].astype('int').astype('category')
AFR_grid['REG']  = AFR_grid['REG'].astype('int').astype('category')
AFR_grid

In [ ]:
RANDOM_STATE = 42
pipeline_dict = {}
grid_dict = {}
best_features = [ 16, 15, 14, 13, 12, 11,10,
                 9,8,7,6,5,4
                 ]

labels = ['RFE_16', 'RFE_15', 'RFE_14', 'RFE_13', 'RFE_12', 'RFE_11', 'RFE_10', 'RFE_09', 'RFE_08', 'RFE_07', 
    'RFE_06', 'RFE_05', 'RFE_04', 
]

#tuned_params = {item.replace('ttr__regressor__', 'ttr__regressor__estimator__'): best_params[item] for item in best_params}

for best_feature, label in tqdm_notebook(
    zip(best_features, labels), total=len(labels), desc='Training: '):
    
    print(best_feature)
    
    # this step gurantee consistency in feature ranking
    feature_importance.sort_values(by="RI", ascending=False, inplace=True)
    features_reduced = feature_importance['OBS_REF'].values[0:best_feature]
    
    numeric_transformer = PowerTransformer(method='yeo-johnson', standardize=True)
    categorical_transformer = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

    preprocessor = ColumnTransformer(
        transformers=[
               ("num", numeric_transformer, selector(dtype_exclude="category")),
               ("cat", categorical_transformer, selector(dtype_include="category")),
        ], 
        #remainder='passthrough', verbose_feature_names_out=False
    )


    regressor = RandomForestRegressor(random_state=RANDOM_STATE)


    steps=[("preprocessor", preprocessor), 
           #("regressor", regressor), 
           ('ttr', TransformedTargetRegressor(regressor=regressor, transformer=numeric_transformer))
          ]

    # Initialize Pipeline object
    pipeline = Pipeline(steps=steps)

    pipeline.set_params(**best_params)


  
    trained_model = pipeline.fit(X_afr[features_reduced], y_afr )
    
   

    ######### save

   
    pipeline_dict[f'{label}'] = trained_model
    grid_dict[f'{label}'] = AFR_grid[features_reduced]
    

    print(f'RFE_{best_feature} fitting is terminated' )


In [ ]:
feature_importance.sort_values(by="RI", ascending=False, inplace=True)
RF_features_df = pd.DataFrame({'Labels':feature_importance.index})
RF_features_df['FI'] = feature_importance['RI'].values

for key in pipeline_dict.keys(): 
    RF_features_df[f'{key}'] = pd.Series(pipeline_dict[key]['ttr'].regressor_.feature_importances_).round(2).sort_values()
RF_features_df

In [ ]:
predictions_df = {}
xs = range(len(ds_afr.X))
ys = range(len(ds_afr.Y))
nn = (len(ds_afr.Y), len( ds_afr.X))


hq_gt = Afr_OD_rab[[grid_index_afr, target]].set_index(grid_index_afr)
hq_gt.index.names = ['index']

for key, pipeline  in tqdm_notebook(pipeline_dict.items() , 
                                            desc=f'Modelling: '):
    print(key)
    pipeline = pipeline_dict[key]
    AFR_grid = grid_dict[key]
    AFR_Q_RFR = np.zeros(nn) # predicted HF value
    predictions_df[key] = pd.DataFrame({'X': ds_afr.XV.values.ravel(), 'Y': ds_afr.YV.values.ravel()})

    predictions_df[key]['Prediction'] = pipeline.predict(AFR_grid).reshape(-1,1)
    predictions_df[key].index.names = ['index']
    final_df = pd.merge(predictions_df[key], hq_gt,  how="left", on="index")
    final_df.to_csv(DIR/'Grids'/'Outputs'/f'{key}_rab.xyz' , index=False, header=True, sep='\t')
    ds_afr[key] = (('Y', 'X'), predictions_df[key]['Prediction'].values.reshape(nn) )
    
ds_afr.to_netcdf(DIR/'Grids'/'Outputs'/"ds_afr_rab.nc", mode='w', 
                    engine='netcdf4')

In [ ]:
for label in labels:
    diff = ds_afr['RFE_16'] - ds_afr[label]
    print(f'Difference between 16 and {label}: {np.round(diff.mean(), 2)}')
print('##'*10)  
for label in labels:
    diff = ds_afr['RFE_11'] - ds_afr[label]
    print(f'Difference between 11 and {label}: {np.round(diff.mean(), 2)}')

# RFECV

In [ ]:
sub_figs = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')

#create two datsets for A ratibng and b ratings
hq_f = DIR / 'Dataset'/ 'References'/'q_Heat_Flow'/'NGHF.csv'

elev_cut = -1000

record_total = pd.read_csv(hq_f)

record_total = record_total.rename(columns={'heat-flow (mW/m2)': target, 'longitude': 'lon', 'latitude': 'lat'})






hq_afr_lat = record_total[record_total['lat'].between(afr_lat_min, afr_lat_max, inclusive='both')]
record_total_afr = hq_afr_lat[hq_afr_lat['lon'].between(afr_lon_min, afr_lon_max, inclusive='both')]



hq_no_pole_afr = record_total_afr.dropna(subset = ['lon', 'lat', target])

hq_no_pole_afr = hq_no_pole_afr[hq_no_pole_afr[target].between(hq_lower_bound, hq_upper_bound, inclusive='both')]


#hq_no_pole_afr = hq_clean[hq_clean['lat'].between(world_lat_min, world_lat_max, inclusive='both')]
hq_deep_afr = hq_no_pole_afr[(hq_no_pole_afr['elevation (m)']>elev_cut)][['lon', 'lat',target]]



hq_final_a_afr = hq_no_pole_afr[(hq_no_pole_afr ['code6']=='A') & (hq_no_pole_afr ['elevation (m)']>elev_cut)][['lon', 'lat',target]]
hq_final_b_afr= hq_no_pole_afr[(hq_no_pole_afr ['code6']=='B') & (hq_no_pole_afr ['elevation (m)']>elev_cut)][['lon', 'lat',target]]





In [ ]:
Best_feature = 11

obs = obs.reset_index(drop=False).set_index('LABELS')

reduced_datsets = obs.loc[feature_importance.iloc[0:Best_feature, 0].index, :]
reduced_datsets = reduced_datsets.reset_index()




obs = obs.reset_index(drop=False).set_index('OBS_AFR')

reduced_datsets

In [ ]:


#proj=f"l135/-25/-45/-10/2c"
region_gmt= [afr_lon_min, afr_lon_max,afr_lat_min, afr_lat_max]
projection = 'M5.4i'


frames = ['wNes', 'wNes','wNEs','Wnes', 'wnes', 'wnes','wnEs','WneS', 'wneS', 'wneS','wnES','wnES']

fig = pygmt.Figure()



pygmt.config(
FONT='30p',
MAP_TICK_PEN='1p',
FONT_ANNOT='26p',
#MAP_GRID_PEN  ,
#MAP_TICK_LENGTH
#MAP_TICK_PEN
)

#pygmt.config(FONT='30p')

with fig.subplot(
    nrows=6,
    ncols=4,
    #figsize=("30c", "15c"),  # width of 15 cm, height of 6 cm
    subsize = ("13.5c", "13.5c"),
    autolabel=['A)+o0.3/-1.5'],
    margins=["0.3c", "3.5c"],  # horizontal 0.3 cm and vertical 0.2 cm margins
    #title=model[3:],
    #sharex="bt",  # shared x-axis on the bottom side
    sharey=True,  # shared y-axis on the left side
    #frame="WSrt",
):

    with fig.set_panel(panel=0):

        cmap = pygmt.makecpt(
                cmap=DIR/'GMT'/'temperature.cpt', #temp 19lev
            #cmap='lajolla',
                series='30/130',
                #truncate = '40/140',
                #continuous=True,
                reverse=False,
            )

        fig.basemap(region=region_gmt, projection=projection, frame='WNes', panel=0)

      


        fig.coast(
            projection=projection,
            #shorelines=0.5,
            water="lightblue", 
            shorelines="0.1p,black",
            borders=["1/0.001p,black"],
            lakes="lightblue",
            rivers="lightblue" ,
            land='darkgrey'
            #borders=["1/0.5p,black"],
            #water='white',
            
            )

        fig.plot(x=Afr_OD_rab.lon, y=Afr_OD_rab.lat,  cmap=True, projection=projection,
             color=Afr_OD_rab[target], #label=f"'A Rating'",
                      pen="0.01p,darkgrey", style="c0.21c")
        fig.colorbar(frame=["af", f"x+lGHF\t\t[mW/m@+2@+]"], 
                     position=f"g{str(afr_lon_min-1)}/{str(afr_lat_min-10)}+w12.5c/0.5c+h+e")
    

    for  cmap_i, grid_label_i, v_range_i, label_i ,unit_i, importance, panel,  in zip(
         reduced_datsets['CMAPS'],  reduced_datsets['OBS_AFR'],
                    reduced_datsets['V_RANGE_AFR'], reduced_datsets['LABELS_gmt'],
        reduced_datsets['UNITS_gmt'], feature_importance['RI'],
        list(range(len(reduced_datsets)))):

        
        with fig.set_panel(panel=panel+1):
            if cmap_i =='bilbao':
                pygmt.makecpt(
                cmap=cmap_i, #temp 19lev
                series=f'{v_range_i[0]}/{v_range_i[1]}',
                #continuous=True,
                reverse=True,
                )
            elif grid_label_i in ['REG', 'GLIM']:
                pygmt.makecpt(
                cmap=cmap_i, #temp 19lev
                series=f'{v_range_i[0]}/{v_range_i[1]}/1',
                #categorical=True,
                #continuous=True,
                )
            elif grid_label_i in ['SV_Velocity', 'PV_Velocity']:
                pygmt.makecpt(
                cmap=cmap_i, #temp 19lev
                series=f'{ds_afr[grid_label_i].min().values}/{ds_afr[grid_label_i].max().values}',
                #continuous=True,
                reverse=False,
                )
            elif grid_label_i == 'EMAG2':
                pygmt.makecpt(
                cmap=cmap_i, #temp 19lev
                series=f'{v_range_i[0]}/{v_range_i[1]}',
                #continuous=True,
                reverse=False,
                )
            else:
                pygmt.makecpt(
                cmap=cmap_i, #temp 19lev
                series=f'{v_range_i[0]}/{v_range_i[1]}',
                #continuous=True,
                reverse=False,
                )
                

            fig.basemap(region=region_gmt, projection=projection, frame=frames[panel], panel=panel+1)



            fig.grdimage(
                 grid=ds_afr[grid_label_i], # xarray.DataArray containing VSV values
                 region=region_gmt,
                 projection=projection,
                     cmap=cmap,

                #shading='+a45+nt0.5'
                #shading=dgrid
            )





            fig.coast(
                projection=projection,
                #shorelines=0.5,
                water="lightblue", 
                shorelines="0.1p,black",
                borders=["1/0.001p,black"],
                lakes="lightblue",
                rivers="lightblue" ,
                #borders=["1/0.5p,black"],
                #water='white',
                )

            #print(f'{importance} {label_i}')
            #fig.colorbar(frame=["af", f'x+l"{label_i}"\t\t"[{unit_i}]"\t\t({str(round(importance*100,1))}\%)',
            fig.colorbar(frame=["afg+el",f'x+l"{label_i}"\t\t"[{unit_i}]"',],
                    position=f"g{str(afr_lon_min-1)}/{str(afr_lat_min-10)}+w12.5c/0.5c+h+e")
            

 

fig.show(width=800)


fig.savefig(DIR/'fig'/"fig_3.pdf", dpi=300 )
fig.savefig(DIR/'fig'/"fig_3.jpg", dpi=300 )